In [11]:
from utils import max_features, maxlen, embed_size_fastText, embed_size_glove, embed_size_glove_twitter
from utils import batch_size, epochs, run_5fold, write_prediction, eval_results
from model import get_CNN_model
import pickle
import pandas as pd

In [2]:
fastText = pickle.load(open("input/fastText.300d.pickle", "rb"))
glove = pickle.load(open("input/glove.300d.pickle", "rb"))
glove_twitter = pickle.load(open("input/glove.twitter.200d.pickle", "rb"))

In [3]:
x_train = pickle.load(open("input/x_train.pickle", "rb"))
y_train = pickle.load(open("input/y_train.pickle", "rb"))
x_test = pickle.load(open("input/x_test.pickle", "rb"))
submission = pd.read_csv("input/sample_submission.csv")

## check different embedding

In [9]:
params = {
    "maxlen": maxlen,
    "max_features": max_features,
    "dropout": 0.2,
    "num_filter": 50,
    "kernel_size": 3,
    "reg": 0.0
}

In [5]:
embeddings = [(fastText, embed_size_fastText, "fastText"), (glove, embed_size_glove, "glove"), (glove_twitter, embed_size_glove_twitter, "glove_twitter")]

In [6]:
predictions = {}

In [10]:
for embed in embeddings:
    print ("\n********************{}********************".format(embed[2]))
    params["embedding_matrix"] = embed[0]
    params["embed_size"] = embed[1]
    y_preds = run_5fold(x_train, y_train, x_test, "loss", get_CNN_model, **params)
    predictions[embed[2]] = y_preds


********************fastText********************
Train on 127656 samples, validate on 31915 samples
Epoch 1/50
 - 29s - loss: 0.0697 - acc: 0.9760 - auc_roc: 0.8875 - val_loss: 0.0483 - val_acc: 0.9817 - val_auc_roc: 0.9624
Epoch 2/50
 - 29s - loss: 0.0424 - acc: 0.9841 - auc_roc: 0.9720 - val_loss: 0.0454 - val_acc: 0.9825 - val_auc_roc: 0.9771
Epoch 3/50
 - 29s - loss: 0.0350 - acc: 0.9865 - auc_roc: 0.9803 - val_loss: 0.0463 - val_acc: 0.9827 - val_auc_roc: 0.9824
Epoch 00003: early stopping
val loss: 0.04627582911981529
val acc: 0.9827301460337381
val auc roc: 0.9823546823860615

Train on 127657 samples, validate on 31914 samples
Epoch 1/50
 - 29s - loss: 0.0645 - acc: 0.9784 - auc_roc: 0.9056 - val_loss: 0.0478 - val_acc: 0.9819 - val_auc_roc: 0.9683
Epoch 2/50
 - 29s - loss: 0.0416 - acc: 0.9842 - auc_roc: 0.9758 - val_loss: 0.0455 - val_acc: 0.9824 - val_auc_roc: 0.9799
Epoch 3/50
 - 29s - loss: 0.0344 - acc: 0.9866 - auc_roc: 0.9825 - val_loss: 0.0456 - val_acc: 0.9829 - val_a

Epoch 2/50
 - 21s - loss: 0.0433 - acc: 0.9836 - auc_roc: 0.9771 - val_loss: 0.0443 - val_acc: 0.9832 - val_auc_roc: 0.9803
Epoch 3/50
 - 21s - loss: 0.0378 - acc: 0.9853 - auc_roc: 0.9825 - val_loss: 0.0446 - val_acc: 0.9833 - val_auc_roc: 0.9839
Epoch 00003: early stopping
val loss: 0.04464892064548453
val acc: 0.9832988464384813
val auc roc: 0.9839194666445087



In [12]:
eval_results(predictions)

key: fastText
loss: 0.04609971681698133 (+/- 0.0012580884288926478)
acc: 0.9828372957186456 (+/- 0.0002604796014717037)
roc_auc: 0.9835281159336994 (+/- 0.0008054109379638884)



key: glove
loss: 0.04559248425298761 (+/- 0.00066710734655799)
acc: 0.9827965611517264 (+/- 0.0003516438407890546)
roc_auc: 0.9848998829240934 (+/- 0.0006772049605117584)



key: glove_twitter
loss: 0.045302061441733474 (+/- 0.0007931266631046235)
acc: 0.9829229399625717 (+/- 0.0004347512370955112)
roc_auc: 0.9839322949097496 (+/- 0.0010873170500473907)





##### result
- fastText: 

loss: 0.04609971681698133 (+/- 0.0012580884288926478)

acc: 0.9828372957186456 (+/- 0.0002604796014717037)

roc_auc: 0.9835281159336994 (+/- 0.0008054109379638884)

- glove: 

loss: 0.04559248425298761 (+/- 0.00066710734655799)

acc: 0.9827965611517264 (+/- 0.0003516438407890546)

roc_auc: 0.9848998829240934 (+/- 0.0006772049605117584)

- glove.twitter: 

loss: 0.045302061441733474 (+/- 0.0007931266631046235)

acc: 0.9829229399625717 (+/- 0.0004347512370955112)

roc_auc: 0.9839322949097496 (+/- 0.0010873170500473907)

=============================================

- mini conclusion:

multi channel: glove, glove.twitter

In [14]:
opt_glove = predictions['glove']['pred']

In [15]:
opt_twitter = predictions['glove_twitter']['pred']

# single model for glove twitter

## check num_filter
(suggest values: 100~600: test 100, 300, 500)

In [21]:
filters = [100, 300, 500]  # for the result of 50, refer to the previous log

In [18]:
params = {
    "maxlen": maxlen,
    "max_features": max_features,
    "dropout": 0.2,
    "kernel_size": 3,
    "reg": 0.0,
    "embedding_matrix": glove_twitter,
    "embed_size": embed_size_glove_twitter
}

In [19]:
predictions_twitter = {}

In [22]:
for f in filters:
    print ("\n********************f = {}********************".format(f))
    params["num_filter"] = f
    y_pred = run_5fold(x_train, y_train, x_test, "loss", get_CNN_model, **params)
    predictions_twitter[f] = y_pred


********************f = 100********************
Train on 127656 samples, validate on 31915 samples
Epoch 1/50
 - 26s - loss: 0.0629 - acc: 0.9781 - auc_roc: 0.9161 - val_loss: 0.0472 - val_acc: 0.9822 - val_auc_roc: 0.9715
Epoch 2/50
 - 26s - loss: 0.0422 - acc: 0.9839 - auc_roc: 0.9778 - val_loss: 0.0462 - val_acc: 0.9822 - val_auc_roc: 0.9810
Epoch 3/50
 - 26s - loss: 0.0363 - acc: 0.9859 - auc_roc: 0.9832 - val_loss: 0.0440 - val_acc: 0.9836 - val_auc_roc: 0.9847
Epoch 4/50
 - 26s - loss: 0.0315 - acc: 0.9878 - auc_roc: 0.9859 - val_loss: 0.0458 - val_acc: 0.9829 - val_auc_roc: 0.9868
Epoch 00004: early stopping
val loss: 0.04583091041125438
val acc: 0.982871147235247
val auc roc: 0.9867898948435758

Train on 127657 samples, validate on 31914 samples
Epoch 1/50
 - 26s - loss: 0.0634 - acc: 0.9776 - auc_roc: 0.9158 - val_loss: 0.0467 - val_acc: 0.9829 - val_auc_roc: 0.9719
Epoch 2/50
 - 26s - loss: 0.0423 - acc: 0.9840 - auc_roc: 0.9778 - val_loss: 0.0446 - val_acc: 0.9831 - val_auc

Epoch 00003: early stopping
val loss: 0.045533780417572986
val acc: 0.9830325051694837
val auc roc: 0.9863445628830131



In [23]:
eval_results(predictions_twitter)

key: 100
loss: 0.04546199066764082 (+/- 0.001634462343494365)
acc: 0.9830023201919488 (+/- 0.0004395332049064285)
roc_auc: 0.9852713829746189 (+/- 0.0009901750425443203)



key: 300
loss: 0.04588088472710787 (+/- 0.002587987363538152)
acc: 0.9828487878298529 (+/- 0.0009820900745316948)
roc_auc: 0.9867106593074004 (+/- 0.0006931558414526911)



key: 500
loss: 0.045338480186612004 (+/- 0.0012909304583345598)
acc: 0.9829542779099537 (+/- 0.0004968131354375467)
roc_auc: 0.9851644077428029 (+/- 0.0012832848194246622)





###### result: 

- f = 50

loss: 0.045302061441733474 (+/- 0.0007931266631046235)

acc: 0.9829229399625717 (+/- 0.0004347512370955112)

roc_auc: 0.9839322949097496 (+/- 0.0010873170500473907)


- f = 100

loss: 0.04546199066764082 (+/- 0.001634462343494365)

acc: 0.9830023201919488 (+/- 0.0004395332049064285)

roc_auc: 0.9852713829746189 (+/- 0.0009901750425443203)

- f = 300

loss: 0.04588088472710787 (+/- 0.002587987363538152)

acc: 0.9828487878298529 (+/- 0.0009820900745316948)

roc_auc: 0.9867106593074004 (+/- 0.0006931558414526911)

- f = 500

loss: 0.045338480186612004 (+/- 0.0012909304583345598)

acc: 0.9829542779099537 (+/- 0.0004968131354375467)

roc_auc: 0.9851644077428029 (+/- 0.0012832848194246622)

In [24]:
opt_twitter = predictions['glove_twitter']['pred']

## check kernel_size
(suggest value: [3, 5, 7, 9])

In [25]:
params = {
    "maxlen": maxlen,
    "max_features": max_features,
    "dropout": 0.2,
    "num_filter": 50,
    "reg": 0.0,
    "embedding_matrix": glove_twitter,
    "embed_size": embed_size_glove_twitter
}

In [26]:
kernels = [5, 7, 9]  # for 3, see result above

In [27]:
for k in kernels:
    print ("\n********************k = {}********************".format(k))
    params["kernel_size"] = k
    result = run_5fold(x_train, y_train, x_test, "loss", get_CNN_model, **params)    
    predictions_twitter[k] = result


********************k = 5********************
Train on 127656 samples, validate on 31915 samples
Epoch 1/50
 - 24s - loss: 0.0649 - acc: 0.9776 - auc_roc: 0.9187 - val_loss: 0.0477 - val_acc: 0.9823 - val_auc_roc: 0.9696
Epoch 2/50
 - 24s - loss: 0.0433 - acc: 0.9838 - auc_roc: 0.9761 - val_loss: 0.0454 - val_acc: 0.9828 - val_auc_roc: 0.9798
Epoch 3/50
 - 24s - loss: 0.0372 - acc: 0.9858 - auc_roc: 0.9821 - val_loss: 0.0451 - val_acc: 0.9829 - val_auc_roc: 0.9837
Epoch 4/50
 - 24s - loss: 0.0321 - acc: 0.9875 - auc_roc: 0.9851 - val_loss: 0.0475 - val_acc: 0.9829 - val_auc_roc: 0.9859
Epoch 00004: early stopping
val loss: 0.04753291173478419
val acc: 0.9828868136106381
val auc roc: 0.9858642352574226

Train on 127657 samples, validate on 31914 samples
Epoch 1/50
 - 24s - loss: 0.0654 - acc: 0.9778 - auc_roc: 0.9087 - val_loss: 0.0485 - val_acc: 0.9821 - val_auc_roc: 0.9686
Epoch 2/50
 - 24s - loss: 0.0430 - acc: 0.9839 - auc_roc: 0.9756 - val_loss: 0.0470 - val_acc: 0.9823 - val_auc_

Epoch 00003: early stopping
val loss: 0.046753579186505675
val acc: 0.9823588156166729
val auc roc: 0.9831283277883157

Train on 127657 samples, validate on 31914 samples
Epoch 1/50
 - 28s - loss: 0.0635 - acc: 0.9783 - auc_roc: 0.9198 - val_loss: 0.0482 - val_acc: 0.9820 - val_auc_roc: 0.9701
Epoch 2/50
 - 28s - loss: 0.0422 - acc: 0.9841 - auc_roc: 0.9767 - val_loss: 0.0456 - val_acc: 0.9829 - val_auc_roc: 0.9801
Epoch 3/50
 - 28s - loss: 0.0352 - acc: 0.9865 - auc_roc: 0.9825 - val_loss: 0.0469 - val_acc: 0.9828 - val_auc_roc: 0.9839
Epoch 00003: early stopping
val loss: 0.04687114304075521
val acc: 0.9828497244651779
val auc roc: 0.9838933663319815



In [28]:
eval_results(predictions_twitter)

key: 100
loss: 0.04546199066764082 (+/- 0.001634462343494365)
acc: 0.9830023201919488 (+/- 0.0004395332049064285)
roc_auc: 0.9852713829746189 (+/- 0.0009901750425443203)



key: 300
loss: 0.04588088472710787 (+/- 0.002587987363538152)
acc: 0.9828487878298529 (+/- 0.0009820900745316948)
roc_auc: 0.9867106593074004 (+/- 0.0006931558414526911)



key: 500
loss: 0.045338480186612004 (+/- 0.0012909304583345598)
acc: 0.9829542779099537 (+/- 0.0004968131354375467)
roc_auc: 0.9851644077428029 (+/- 0.0012832848194246622)



key: 5
loss: 0.04606231503054052 (+/- 0.0010097946382350116)
acc: 0.9829166728211532 (+/- 0.0003102764266677338)
roc_auc: 0.9844617126249698 (+/- 0.001180167411775649)



key: 7
loss: 0.04629628242034946 (+/- 0.0014595847765578977)
acc: 0.9829386089449994 (+/- 0.0003367164145233068)
roc_auc: 0.9841194321166867 (+/- 0.0015817458364257356)



key: 9
loss: 0.04673610524711422 (+/- 0.0008944812529786167)
acc: 0.9827704502664698 (+/- 0.0002996998346054342)
roc_auc: 0.983723342587

##### result:

- k = 3

loss: 0.045302061441733474 (+/- 0.0007931266631046235)

acc: 0.9829229399625717 (+/- 0.0004347512370955112)

roc_auc: 0.9839322949097496 (+/- 0.0010873170500473907)

- k = 5

loss: 0.04606231503054052 (+/- 0.0010097946382350116)

acc: 0.9829166728211532 (+/- 0.0003102764266677338)

roc_auc: 0.9844617126249698 (+/- 0.001180167411775649)

- k = 7

loss: 0.04629628242034946 (+/- 0.0014595847765578977)

acc: 0.9829386089449994 (+/- 0.0003367164145233068)

roc_auc: 0.9841194321166867 (+/- 0.0015817458364257356)


- k = 9

loss: 0.04673610524711422 (+/- 0.0008944812529786167)

acc: 0.9827704502664698 (+/- 0.0002996998346054342)

roc_auc: 0.9837233425877759 (+/- 0.0009990948818940741)

In [29]:
opt_twitter = opt_twitter

## check drop out
(suggest value 0.1~0.5: test 0.1, 0.2, 0.5)

In [30]:
params = {
    "maxlen": maxlen,
    "max_features": max_features,
    "kernel_size": 3,
    "num_filter": 50,
    "reg": 0.0,
    "embedding_matrix": glove_twitter,
    "embed_size": embed_size_glove_twitter
}

In [31]:
dropouts = [0.1, 0.5] # 0.2 see previous result

In [32]:
for d in dropouts:
    print ("\n********************d = {}********************".format(d))
    params["dropout"] = d
    result = run_5fold(x_train, y_train, x_test, "loss", get_CNN_model, **params)    
    predictions_twitter[d] = result


********************d = 0.1********************
Train on 127656 samples, validate on 31915 samples
Epoch 1/50
 - 22s - loss: 0.0614 - acc: 0.9788 - auc_roc: 0.9213 - val_loss: 0.0465 - val_acc: 0.9823 - val_auc_roc: 0.9741
Epoch 2/50
 - 21s - loss: 0.0415 - acc: 0.9842 - auc_roc: 0.9797 - val_loss: 0.0447 - val_acc: 0.9830 - val_auc_roc: 0.9825
Epoch 3/50
 - 21s - loss: 0.0353 - acc: 0.9863 - auc_roc: 0.9844 - val_loss: 0.0456 - val_acc: 0.9826 - val_auc_roc: 0.9857
Epoch 00003: early stopping
val loss: 0.04559372121892191
val acc: 0.9826361501700598
val auc roc: 0.9857020561642432

Train on 127657 samples, validate on 31914 samples
Epoch 1/50
 - 21s - loss: 0.0700 - acc: 0.9748 - auc_roc: 0.8973 - val_loss: 0.0473 - val_acc: 0.9824 - val_auc_roc: 0.9681
Epoch 2/50
 - 21s - loss: 0.0424 - acc: 0.9840 - auc_roc: 0.9755 - val_loss: 0.0454 - val_acc: 0.9828 - val_auc_roc: 0.9796
Epoch 3/50
 - 21s - loss: 0.0361 - acc: 0.9862 - auc_roc: 0.9821 - val_loss: 0.0456 - val_acc: 0.9832 - val_au

In [33]:
eval_results(predictions_twitter)

key: 100
loss: 0.04546199066764082 (+/- 0.001634462343494365)
acc: 0.9830023201919488 (+/- 0.0004395332049064285)
roc_auc: 0.9852713829746189 (+/- 0.0009901750425443203)



key: 300
loss: 0.04588088472710787 (+/- 0.002587987363538152)
acc: 0.9828487878298529 (+/- 0.0009820900745316948)
roc_auc: 0.9867106593074004 (+/- 0.0006931558414526911)



key: 500
loss: 0.045338480186612004 (+/- 0.0012909304583345598)
acc: 0.9829542779099537 (+/- 0.0004968131354375467)
roc_auc: 0.9851644077428029 (+/- 0.0012832848194246622)



key: 5
loss: 0.04606231503054052 (+/- 0.0010097946382350116)
acc: 0.9829166728211532 (+/- 0.0003102764266677338)
roc_auc: 0.9844617126249698 (+/- 0.001180167411775649)



key: 7
loss: 0.04629628242034946 (+/- 0.0014595847765578977)
acc: 0.9829386089449994 (+/- 0.0003367164145233068)
roc_auc: 0.9841194321166867 (+/- 0.0015817458364257356)



key: 9
loss: 0.04673610524711422 (+/- 0.0008944812529786167)
acc: 0.9827704502664698 (+/- 0.0002996998346054342)
roc_auc: 0.983723342587

##### result

- d = 0.1

loss: 0.04591581410463604 (+/- 0.0008709015027983505)

acc: 0.9828017841505808 (+/- 0.00041865175710850976)

roc_auc: 0.9850836938253522 (+/- 0.0009719962580997339)

- d = 0.2

loss: 0.045302061441733474 (+/- 0.0007931266631046235)

acc: 0.9829229399625717 (+/- 0.0004347512370955112)

roc_auc: 0.9839322949097496 (+/- 0.0010873170500473907)


- d = 0.5

loss: 0.046194996284758226 (+/- 0.0008584190898952846)

acc: 0.9824863555981682 (+/- 0.0004646613514855763)

roc_auc: 0.9790240309425113 (+/- 0.002612822858693605)

In [34]:
opt_twitter = opt_twitter

#### result params for glove twitter

In [37]:
params = {
    "maxlen": maxlen,
    "max_features": max_features,
    "kernel_size": 3,
    "num_filter": 50,
    "dropout": 0.2,
    "reg": 0.0,
    "embedding_matrix": glove_twitter,
    "embed_size": embed_size_glove_twitter
}

In [36]:
write_prediction(opt_twitter, submission, "output/CNN_twitter.csv")

# single model for glove

## check num_filter
(suggest values: 100~600: test 100, 300, 500)

In [38]:
filters = [100, 300, 500]  # for the result of 50, refer to the previous log

In [39]:
params = {
    "maxlen": maxlen,
    "max_features": max_features,
    "dropout": 0.2,
    "kernel_size": 3,
    "reg": 0.0,
    "embedding_matrix": glove,
    "embed_size": embed_size_glove
}

In [40]:
predictions_glove = {}

In [41]:
for f in filters:
    print ("\n********************f = {}********************".format(f))
    params["num_filter"] = f
    result = run_5fold(x_train, y_train, x_test, "loss", get_CNN_model, **params)    
    predictions_glove[f] = result


********************f = 100********************
Train on 127656 samples, validate on 31915 samples
Epoch 1/50
 - 33s - loss: 0.0579 - acc: 0.9797 - auc_roc: 0.9289 - val_loss: 0.0451 - val_acc: 0.9828 - val_auc_roc: 0.9755
Epoch 2/50
 - 33s - loss: 0.0401 - acc: 0.9848 - auc_roc: 0.9805 - val_loss: 0.0449 - val_acc: 0.9827 - val_auc_roc: 0.9830
Epoch 3/50
 - 33s - loss: 0.0338 - acc: 0.9870 - auc_roc: 0.9850 - val_loss: 0.0447 - val_acc: 0.9830 - val_auc_roc: 0.9861
Epoch 4/50
 - 33s - loss: 0.0275 - acc: 0.9894 - auc_roc: 0.9873 - val_loss: 0.0498 - val_acc: 0.9815 - val_auc_roc: 0.9881
Epoch 00004: early stopping
val loss: 0.04975917948724782
val acc: 0.9815394875604718
val auc roc: 0.988082192087181

Train on 127657 samples, validate on 31914 samples
Epoch 1/50
 - 33s - loss: 0.0595 - acc: 0.9789 - auc_roc: 0.9235 - val_loss: 0.0452 - val_acc: 0.9827 - val_auc_roc: 0.9737
Epoch 2/50
 - 33s - loss: 0.0403 - acc: 0.9845 - auc_roc: 0.9794 - val_loss: 0.0446 - val_acc: 0.9829 - val_auc

In [42]:
eval_results(predictions_glove)

key: 100
loss: 0.04601445630171195 (+/- 0.0020066506932860306)
acc: 0.9827829910552092 (+/- 0.0006525990545582435)
roc_auc: 0.9865255406409961 (+/- 0.0009220619202532495)



key: 300
loss: 0.04493380877026916 (+/- 0.0008816936950137447)
acc: 0.9830409674726788 (+/- 0.0002668991024840636)
roc_auc: 0.9855141711177511 (+/- 0.0015221667521623432)



key: 500
loss: 0.046407338910416165 (+/- 0.00213438906316632)
acc: 0.982989791722234 (+/- 0.00046590775908032697)
roc_auc: 0.9857171781847021 (+/- 0.0011258855863419304)





###### result:

- 50:

loss: 0.04559248425298761 (+/- 0.00066710734655799)

acc: 0.9827965611517264 (+/- 0.0003516438407890546)

roc_auc: 0.9848998829240934 (+/- 0.0006772049605117584)

- 100:

loss: 0.04601445630171195 (+/- 0.0020066506932860306)

acc: 0.9827829910552092 (+/- 0.0006525990545582435)

roc_auc: 0.9865255406409961 (+/- 0.0009220619202532495)

- 300:

loss: 0.04493380877026916 (+/- 0.0008816936950137447)

acc: 0.9830409674726788 (+/- 0.0002668991024840636)

roc_auc: 0.9855141711177511 (+/- 0.0015221667521623432)


- 500:

loss: 0.046407338910416165 (+/- 0.00213438906316632)

acc: 0.982989791722234 (+/- 0.00046590775908032697)

roc_auc: 0.9857171781847021 (+/- 0.0011258855863419304)

In [44]:
opt_glove = predictions_glove[300]['pred']

In [45]:
opt_glove

[array([[9.9988878e-01, 3.2607320e-01, 9.9909186e-01, 4.2837597e-02,
         9.7526181e-01, 5.0506383e-01],
        [5.2075975e-05, 6.7775509e-06, 3.7139853e-05, 8.9647619e-06,
         7.7980643e-05, 2.6870233e-05],
        [5.8451872e-03, 4.8241992e-03, 1.0806988e-02, 3.0471815e-04,
         6.2295264e-03, 1.6877786e-03],
        ...,
        [9.8656739e-05, 6.1778963e-05, 2.6817099e-04, 1.4880783e-04,
         5.4808811e-04, 8.7899913e-05],
        [7.9450890e-04, 1.9844156e-04, 5.7480810e-04, 4.4849576e-04,
         2.3525646e-03, 5.2878205e-03],
        [9.9707162e-01, 6.8134344e-03, 9.5658046e-01, 4.4497019e-03,
         4.3048623e-01, 5.1593492e-03]], dtype=float32),
 array([[9.9936670e-01, 1.5884735e-01, 9.8224074e-01, 1.1089378e-02,
         9.8304409e-01, 7.5670975e-01],
        [8.7981585e-05, 2.0524496e-05, 7.4573647e-05, 4.7461276e-06,
         2.5087112e-04, 2.5516289e-05],
        [1.0546318e-02, 2.5208511e-03, 1.4501218e-03, 6.8583700e-05,
         3.4691619e-03, 2.125

## check kernel size
(suggest value: [3, 5, 7, 9])

In [46]:
params = {
    "maxlen": maxlen,
    "max_features": max_features,
    "dropout": 0.2,
    "num_filter": 300,
    "reg": 0.0,
    "embedding_matrix": glove,
    "embed_size": embed_size_glove
}

In [47]:
kernels = [5, 7, 9]  # for 3, see result above

In [ ]:
for k in kernels:
    print ("\n********************k = {}********************".format(k))
    params["kernel_size"] = k
    result = run_5fold(x_train, y_train, x_test, "loss", get_CNN_model, **params)    
    predictions_glove[k] = result


********************k = 5********************
Train on 127656 samples, validate on 31915 samples
Epoch 1/50
 - 72s - loss: 0.0560 - acc: 0.9799 - auc_roc: 0.9400 - val_loss: 0.0484 - val_acc: 0.9816 - val_auc_roc: 0.9763
Epoch 2/50


In [50]:
eval_results(predictions_glove)

key: 100
loss: 0.04601445630171195 (+/- 0.0020066506932860306)
acc: 0.9827829910552092 (+/- 0.0006525990545582435)
roc_auc: 0.9865255406409961 (+/- 0.0009220619202532495)



key: 300
loss: 0.04493380877026916 (+/- 0.0008816936950137447)
acc: 0.9830409674726788 (+/- 0.0002668991024840636)
roc_auc: 0.9855141711177511 (+/- 0.0015221667521623432)



key: 500
loss: 0.046407338910416165 (+/- 0.00213438906316632)
acc: 0.982989791722234 (+/- 0.00046590775908032697)
roc_auc: 0.9857171781847021 (+/- 0.0011258855863419304)



key: 5
loss: 0.04748997002168405 (+/- 0.0021834484931673925)
acc: 0.9826701848987014 (+/- 0.0005885760699276943)
roc_auc: 0.9865483508044474 (+/- 0.0012091863265703145)



key: 7
loss: 0.04642165413081062 (+/- 0.0016689807822862668)
acc: 0.9828007364559337 (+/- 0.0007606376298816129)
roc_auc: 0.9848486750642831 (+/- 0.0013648982558387946)



key: 9
loss: 0.04623318673644024 (+/- 0.0012161839793471843)
acc: 0.9829438316817724 (+/- 0.00017016266191228596)
roc_auc: 0.9847446051

##### result 

- 3:

loss: 0.04493380877026916 (+/- 0.0008816936950137447)

acc: 0.9830409674726788 (+/- 0.0002668991024840636)

roc_auc: 0.9855141711177511 (+/- 0.0015221667521623432)

- 5: 

loss: 0.04748997002168405 (+/- 0.0021834484931673925)

acc: 0.9826701848987014 (+/- 0.0005885760699276943)

roc_auc: 0.9865483508044474 (+/- 0.0012091863265703145)

- 7: 

loss: 0.04642165413081062 (+/- 0.0016689807822862668)

acc: 0.9828007364559337 (+/- 0.0007606376298816129)

roc_auc: 0.9848486750642831 (+/- 0.0013648982558387946)

- 9:

loss: 0.04623318673644024 (+/- 0.0012161839793471843)

acc: 0.9829438316817724 (+/- 0.00017016266191228596)

roc_auc: 0.9847446051121892 (+/- 0.0014313961638061227)

In [51]:
opt_glove = opt_glove

## check dropout

In [52]:
params = {
    "maxlen": maxlen,
    "max_features": max_features,
    "kernel_size": 3,
    "num_filter": 300,
    "reg": 0.0,
    "embedding_matrix": glove,
    "embed_size": embed_size_glove
}

In [53]:
dropouts = [0.1, 0.5] # 0.2 see previous result

In [55]:
for d in dropouts:
    print ("\n********************d = {}********************".format(d))
    params["dropout"] = d
    result = run_5fold(x_train, y_train, x_test, "loss", get_CNN_model, **params)    
    predictions_glove[d] = result


********************d = 0.1********************
Train on 127656 samples, validate on 31915 samples
Epoch 1/50
 - 57s - loss: 0.0522 - acc: 0.9813 - auc_roc: 0.9485 - val_loss: 0.0434 - val_acc: 0.9833 - val_auc_roc: 0.9802
Epoch 2/50
 - 56s - loss: 0.0374 - acc: 0.9856 - auc_roc: 0.9839 - val_loss: 0.0434 - val_acc: 0.9834 - val_auc_roc: 0.9857
Epoch 00002: early stopping
val loss: 0.043449460532963884
val acc: 0.9834194758740336
val auc roc: 0.9857266164546885

Train on 127657 samples, validate on 31914 samples
Epoch 1/50
 - 56s - loss: 0.0541 - acc: 0.9806 - auc_roc: 0.9427 - val_loss: 0.0440 - val_acc: 0.9831 - val_auc_roc: 0.9788
Epoch 2/50
 - 56s - loss: 0.0375 - acc: 0.9856 - auc_roc: 0.9829 - val_loss: 0.0475 - val_acc: 0.9820 - val_auc_roc: 0.9849
Epoch 00002: early stopping
val loss: 0.047493287397118646
val acc: 0.9819566968121295
val auc roc: 0.984937372867749

Train on 127657 samples, validate on 31914 samples
Epoch 1/50
 - 56s - loss: 0.0544 - acc: 0.9803 - auc_roc: 0.938

In [56]:
eval_results(predictions_glove)

key: 100
loss: 0.04601445630171195 (+/- 0.0020066506932860306)
acc: 0.9827829910552092 (+/- 0.0006525990545582435)
roc_auc: 0.9865255406409961 (+/- 0.0009220619202532495)



key: 300
loss: 0.04493380877026916 (+/- 0.0008816936950137447)
acc: 0.9830409674726788 (+/- 0.0002668991024840636)
roc_auc: 0.9855141711177511 (+/- 0.0015221667521623432)



key: 500
loss: 0.046407338910416165 (+/- 0.00213438906316632)
acc: 0.982989791722234 (+/- 0.00046590775908032697)
roc_auc: 0.9857171781847021 (+/- 0.0011258855863419304)



key: 5
loss: 0.04748997002168405 (+/- 0.0021834484931673925)
acc: 0.9826701848987014 (+/- 0.0005885760699276943)
roc_auc: 0.9865483508044474 (+/- 0.0012091863265703145)



key: 7
loss: 0.04642165413081062 (+/- 0.0016689807822862668)
acc: 0.9828007364559337 (+/- 0.0007606376298816129)
roc_auc: 0.9848486750642831 (+/- 0.0013648982558387946)



key: 9
loss: 0.04623318673644024 (+/- 0.0012161839793471843)
acc: 0.9829438316817724 (+/- 0.00017016266191228596)
roc_auc: 0.9847446051

##### result
- 0.1:

loss: 0.046217885948436246 (+/- 0.0019758657529593882)

acc: 0.9828602680872034 (+/- 0.0006064660632937864)

roc_auc: 0.9862824765004596 (+/- 0.0014912518010662232)

- 0.2:

loss: 0.04493380877026916 (+/- 0.0008816936950137447)

acc: 0.9830409674726788 (+/- 0.0002668991024840636)

roc_auc: 0.9855141711177511 (+/- 0.0015221667521623432)


- 0.5:

loss: 0.04622874744349184 (+/- 0.0014564954240449707)

acc: 0.982428906861531 (+/- 0.0006747995642684885)

roc_auc: 0.9840173709305304 (+/- 0.0012358310243868592)

In [58]:
opt_glove = opt_glove

In [59]:
write_prediction(opt_glove, submission, "output/CNN_glove.csv")

##### params for glove

In [57]:
params = {
    "maxlen": maxlen,
    "max_features": max_features,
    "kernel_size": 3,
    "num_filter": 300,
    "dropout": 0.2,
    "reg": 0.0,
    "embedding_matrix": glove,
    "embed_size": embed_size_glove
}